In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
ipl = pd.read_csv("../data/cleaned/ipl_cleaned.csv")

In [3]:
# Ensure correct order at ball level
ipl = ipl.sort_values(['batter', 'date', 'match_id', 'over', 'ball']).reset_index(drop=True)

# Rolling runs in last 10 balls
ipl['rolling_runs_10'] = (
    ipl.groupby('batter')['batsman_runs']
       .rolling(window=10, min_periods=1)
       .sum()
       .shift(1)
       .reset_index(level=0, drop=True)
)

# Rolling balls (always 10 but handle start)
ipl['rolling_balls_10'] = (
    ipl.groupby('batter')['ball']
       .rolling(window=10, min_periods=1)
       .count()
       .shift(1)
       .reset_index(level=0, drop=True)
)

# Strike Rate last 10 balls
ipl['rolling_strike_rate_10'] = (
    (ipl['rolling_runs_10'] / ipl['rolling_balls_10']) * 100
)

ipl['rolling_strike_rate_10'] = ipl['rolling_strike_rate_10'].fillna(0)


#### Match level data seperation - Batsman

In [5]:
df_batter = ipl.groupby(['match_id','batter','bowler','date','venue','bowling_team']).agg({
    'batsman_runs':'sum',
    'is_wicket':'sum'
}).reset_index()
    
df_batter.head(10)

,match_id,batter,bowler,date,venue,bowling_team,batsman_runs,is_wicket
0,335982,AA Noffke,AB Agarkar,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,2,0
1,335982,AA Noffke,SC Ganguly,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,7,1
2,335982,B Akhil,AB Agarkar,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,0,1
3,335982,BB McCullum,AA Noffke,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,24,0
4,335982,BB McCullum,CL White,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,16,0
5,335982,BB McCullum,JH Kallis,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,32,0
6,335982,BB McCullum,P Kumar,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,32,0
7,335982,BB McCullum,SB Joshi,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,21,0
8,335982,BB McCullum,Z Khan,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bangalore,33,0
9,335982,CL White,AB Agarkar,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,3,1


#### Match level data seperation - Bowler

In [6]:
df_bowler = (
    ipl.groupby(['match_id', 'bowler', 'date', 'venue', 'bowling_team'])
       .agg(
           balls_bowled=('ball', 'count'),
           wickets=('is_wicket', 'sum'),
           runs_conceded=('batsman_runs', 'sum')
       )
       .reset_index()
)


In [7]:
df_bowler['overs_bowled'] = df_bowler['balls_bowled'] / 6


In [8]:
df_bowler = df_bowler.sort_values(['bowler', 'date']).reset_index(drop=True)
df_bowler.head(30)


,match_id,bowler,date,venue,bowling_team,balls_bowled,wickets,runs_conceded,overs_bowled
0,548341,A Ashish Reddy,2012-04-26,Subrata Roy Sahara Stadium,Deccan Chargers,24,2,32,4.000000
1,548346,A Ashish Reddy,2012-04-29,Wankhede Stadium,Deccan Chargers,14,1,10,2.333333
2,548348,A Ashish Reddy,2012-05-01,Barabati Stadium,Deccan Chargers,19,1,31,3.166667
3,548352,A Ashish Reddy,2012-05-04,"MA Chidambaram Stadium, Chepauk",Deccan Chargers,13,1,15,2.166667
4,548356,A Ashish Reddy,2012-05-06,M Chinnaswamy Stadium,Deccan Chargers,25,1,35,4.166667
5,548359,A Ashish Reddy,2012-05-08,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,25,2,36,4.166667
6,548329,A Ashish Reddy,2012-05-10,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,12,0,29,2.000000
7,548373,A Ashish Reddy,2012-05-18,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,12,0,17,2.000000
8,548376,A Ashish Reddy,2012-05-20,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,25,3,24,4.166667
9,598000,A Ashish Reddy,2013-04-05,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,12,1,21,2.000000


In [9]:
df_batter.to_csv("../data/ref/reference.csv",index=False)

In [10]:
# Sort data by batter and date to ensure temporal ordering for all time-based features
df_batter = df_batter.sort_values(['batter','date']).reset_index(drop=True)

In [9]:
df_batter.head()

,match_id,batter,bowler,date,venue,bowling_team,batsman_runs,is_wicket
0,548346,A Ashish Reddy,JEC Franklin,2012-04-29,Wankhede Stadium,Mumbai Indians,3,0
1,548346,A Ashish Reddy,MM Patel,2012-04-29,Wankhede Stadium,Mumbai Indians,0,1
2,548346,A Ashish Reddy,RJ Peterson,2012-04-29,Wankhede Stadium,Mumbai Indians,7,0
3,548352,A Ashish Reddy,BW Hilfenhaus,2012-05-04,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,2,1
4,548352,A Ashish Reddy,DJ Bravo,2012-05-04,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,1,0


##### Calculation of rolling average


In [11]:
ipl =  ipl.sort_values(['batter','match_id','over','ball'])

df_batter['rolling_avg_5'] = (
    df_batter.groupby('batter')['batsman_runs']
      .rolling(window=5, min_periods=1)
      .mean()
      .shift(1)
      .reset_index(level=0, drop=True)
)   

print(df_batter.head(10))

   match_id          batter         bowler        date  \
0    548346  A Ashish Reddy   JEC Franklin  2012-04-29   
1    548346  A Ashish Reddy       MM Patel  2012-04-29   
2    548346  A Ashish Reddy    RJ Peterson  2012-04-29   
3    548352  A Ashish Reddy  BW Hilfenhaus  2012-05-04   
4    548352  A Ashish Reddy       DJ Bravo  2012-05-04   
5    548359  A Ashish Reddy        P Awana  2012-05-08   
6    548359  A Ashish Reddy        P Kumar  2012-05-08   
7    548359  A Ashish Reddy      PP Chawla  2012-05-08   
8    548373  A Ashish Reddy        SW Tait  2012-05-18   
9    548376  A Ashish Reddy  R Vinay Kumar  2012-05-20   

                                       venue                 bowling_team  \
0                           Wankhede Stadium               Mumbai Indians   
1                           Wankhede Stadium               Mumbai Indians   
2                           Wankhede Stadium               Mumbai Indians   
3            MA Chidambaram Stadium, Chepauk         

##### Calculation of venue average


In [12]:

df_batter['venue_avg'] = (
    df_batter.groupby(['batter', 'venue'])['batsman_runs']
      .expanding()
      .mean()
      .shift(1)
      .reset_index(level=[0,1], drop=True)
)

df_batter['venue_avg'] = df_batter['venue_avg'].fillna(0)

print(df_batter.head(5))

   match_id          batter         bowler        date  \
0    548346  A Ashish Reddy   JEC Franklin  2012-04-29   
1    548346  A Ashish Reddy       MM Patel  2012-04-29   
2    548346  A Ashish Reddy    RJ Peterson  2012-04-29   
3    548352  A Ashish Reddy  BW Hilfenhaus  2012-05-04   
4    548352  A Ashish Reddy       DJ Bravo  2012-05-04   

                             venue         bowling_team  batsman_runs  \
0                 Wankhede Stadium       Mumbai Indians             3   
1                 Wankhede Stadium       Mumbai Indians             0   
2                 Wankhede Stadium       Mumbai Indians             7   
3  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             2   
4  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             1   

   is_wicket  rolling_avg_5  venue_avg  
0          0            NaN   4.666667  
1          1       3.000000   3.000000  
2          0       1.500000   1.500000  
3          1       3.333333   7.000000  
4      

##### Calculation of player-to-team average


In [13]:
df_batter['pvt_avg'] = (
    df_batter.groupby(['batter', 'bowling_team'])['batsman_runs']
      .expanding()
      .mean()
      .shift(1) 
      .reset_index(level=[0,1], drop=True)
)

df_batter['pvt_avg'] = df_batter['pvt_avg'].fillna(0)
print(df_batter.head(5))

   match_id          batter         bowler        date  \
0    548346  A Ashish Reddy   JEC Franklin  2012-04-29   
1    548346  A Ashish Reddy       MM Patel  2012-04-29   
2    548346  A Ashish Reddy    RJ Peterson  2012-04-29   
3    548352  A Ashish Reddy  BW Hilfenhaus  2012-05-04   
4    548352  A Ashish Reddy       DJ Bravo  2012-05-04   

                             venue         bowling_team  batsman_runs  \
0                 Wankhede Stadium       Mumbai Indians             3   
1                 Wankhede Stadium       Mumbai Indians             0   
2                 Wankhede Stadium       Mumbai Indians             7   
3  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             2   
4  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             1   

   is_wicket  rolling_avg_5  venue_avg  pvt_avg  
0          0            NaN   4.666667      3.4  
1          1       3.000000   3.000000      3.0  
2          0       1.500000   1.500000      1.5  
3          1

##### Calculation of career avg

In [14]:
df_batter['career_avg'] = df_batter.groupby('batter')['batsman_runs'] \
                     .expanding() \
                     .mean() \
                     .shift(1) \
                     .reset_index(level=0, drop=True)

# Handle NaN values from shift operation (first match for each player)
df_batter['career_avg'] = df_batter['career_avg'].fillna(0)

print(df_batter.head(5))

   match_id          batter         bowler        date  \
0    548346  A Ashish Reddy   JEC Franklin  2012-04-29   
1    548346  A Ashish Reddy       MM Patel  2012-04-29   
2    548346  A Ashish Reddy    RJ Peterson  2012-04-29   
3    548352  A Ashish Reddy  BW Hilfenhaus  2012-05-04   
4    548352  A Ashish Reddy       DJ Bravo  2012-05-04   

                             venue         bowling_team  batsman_runs  \
0                 Wankhede Stadium       Mumbai Indians             3   
1                 Wankhede Stadium       Mumbai Indians             0   
2                 Wankhede Stadium       Mumbai Indians             7   
3  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             2   
4  MA Chidambaram Stadium, Chepauk  Chennai Super Kings             1   

   is_wicket  rolling_avg_5  venue_avg  pvt_avg  career_avg  
0          0            NaN   4.666667      3.4    0.000000  
1          1       3.000000   3.000000      3.0    3.000000  
2          0       1.50000

In [15]:
# first compute at ball level
ipl['pvp_avg_ball'] = (
    ipl.groupby(['batter', 'bowler'])['batsman_runs']
       .expanding()
       .mean()
       .shift(1)
       .reset_index(level=[0,1], drop=True)
)

ipl['pvp_avg_ball'] = ipl['pvp_avg_ball'].fillna(0)

# convert to match level
pvp_match = (
    ipl.groupby(['batter', 'match_id'])['pvp_avg_ball']
       .mean()
       .reset_index()
)

# Remove column if it exists before merging
if 'pvp_avg' in df_batter.columns:
    df_batter = df_batter.drop(columns=['pvp_avg'])
if 'pvp_avg_ball' in df_batter.columns:
    df_batter = df_batter.drop(columns=['pvp_avg_ball'])


# merge into df
df_batter = df_batter.merge(pvp_match, on=['batter','match_id'], how='left')
df_batter.rename(columns={'pvp_avg_ball':'pvp_avg'}, inplace=True)

print(df_batter.head(30))

    match_id          batter           bowler        date  \
0     548346  A Ashish Reddy     JEC Franklin  2012-04-29   
1     548346  A Ashish Reddy         MM Patel  2012-04-29   
2     548346  A Ashish Reddy      RJ Peterson  2012-04-29   
3     548352  A Ashish Reddy    BW Hilfenhaus  2012-05-04   
4     548352  A Ashish Reddy         DJ Bravo  2012-05-04   
5     548359  A Ashish Reddy          P Awana  2012-05-08   
6     548359  A Ashish Reddy          P Kumar  2012-05-08   
7     548359  A Ashish Reddy        PP Chawla  2012-05-08   
8     548373  A Ashish Reddy          SW Tait  2012-05-18   
9     548376  A Ashish Reddy    R Vinay Kumar  2012-05-20   
10    598000  A Ashish Reddy         AB Dinda  2013-04-05   
11    598000  A Ashish Reddy          B Kumar  2013-04-05   
12    598004  A Ashish Reddy         M Kartik  2013-04-07   
13    598004  A Ashish Reddy    R Vinay Kumar  2013-04-07   
14    598048  A Ashish Reddy    R Vinay Kumar  2013-04-09   
15    598048  A Ashish R

In [16]:
# Convert rolling_strike_rate_10 to match level
strike_match = (
    ipl.groupby(['batter', 'match_id'])['rolling_strike_rate_10']
       .mean()
       .reset_index()
)

df_batter = df_batter.merge(strike_match, on=['batter', 'match_id'], how='left')

df_batter['rolling_strike_rate_10'] = df_batter['rolling_strike_rate_10'].fillna(0)


In [ ]:
# # Remove any existing total_match_runs columns before creating again
# cols_to_remove = [
#     'total_match_runs',
#     '
# 
#_x',
#     'total_match_runs_y'
# ]

# df_batter = df_batter.drop(
#     columns=[col for col in cols_to_remove if col in df_batter.columns]
# )


In [ ]:


# total_runs = (
#     df_batter.groupby(['match_id', 'batter'])['batsman_runs']
#       .sum()
#       .reset_index()
#       .rename(columns={'batsman_runs': 'total_match_runs'})
# )

# # Step 2
# df_batter = df_batter.merge(
#     total_runs,
#     on=['match_id', 'batter'],
#     how='left'
# )

# # Step 3
# df_batter['total_match_runs'] = df_batter['total_match_runs'].fillna(0)
# print(df_batter.head(10))


   match_id          batter         bowler        date  \
0    548346  A Ashish Reddy   JEC Franklin  2012-04-29   
1    548346  A Ashish Reddy       MM Patel  2012-04-29   
2    548346  A Ashish Reddy    RJ Peterson  2012-04-29   
3    548352  A Ashish Reddy  BW Hilfenhaus  2012-05-04   
4    548352  A Ashish Reddy       DJ Bravo  2012-05-04   
5    548359  A Ashish Reddy        P Awana  2012-05-08   
6    548359  A Ashish Reddy        P Kumar  2012-05-08   
7    548359  A Ashish Reddy      PP Chawla  2012-05-08   
8    548373  A Ashish Reddy        SW Tait  2012-05-18   
9    548376  A Ashish Reddy  R Vinay Kumar  2012-05-20   

                                       venue                 bowling_team  \
0                           Wankhede Stadium               Mumbai Indians   
1                           Wankhede Stadium               Mumbai Indians   
2                           Wankhede Stadium               Mumbai Indians   
3            MA Chidambaram Stadium, Chepauk         

##### Target value(**)


In [17]:
df_batter['next_match_runs'] = df_batter.groupby('batter')['batsman_runs'].shift(-1)
df_batter = df_batter.dropna()
df_batter.head(10)

,match_id,batter,bowler,date,venue,bowling_team,batsman_runs,is_wicket,rolling_avg_5,venue_avg,pvt_avg,career_avg,pvp_avg,rolling_strike_rate_10,next_match_runs
1,548346,A Ashish Reddy,MM Patel,2012-04-29,Wankhede Stadium,Mumbai Indians,0,1,3.000000,3.000000,3.000000,3.000000,0.685606,67.968254,7.0
2,548346,A Ashish Reddy,RJ Peterson,2012-04-29,Wankhede Stadium,Mumbai Indians,7,0,1.500000,1.500000,1.500000,1.500000,0.685606,67.968254,2.0
3,548352,A Ashish Reddy,BW Hilfenhaus,2012-05-04,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,2,1,3.333333,7.000000,0.000000,3.333333,1.733333,113.333333,1.0
4,548352,A Ashish Reddy,DJ Bravo,2012-05-04,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,1,0,3.000000,2.000000,2.000000,3.000000,1.733333,113.333333,0.0
5,548359,A Ashish Reddy,P Awana,2012-05-08,"Rajiv Gandhi International Stadium, Uppal",Kings XI Punjab,0,1,2.600000,7.333333,4.500000,2.600000,0.708333,100.000000,6.0
6,548359,A Ashish Reddy,P Kumar,2012-05-08,"Rajiv Gandhi International Stadium, Uppal",Kings XI Punjab,6,0,2.000000,0.000000,0.000000,2.166667,0.708333,100.000000,2.0
7,548359,A Ashish Reddy,PP Chawla,2012-05-08,"Rajiv Gandhi International Stadium, Uppal",Kings XI Punjab,2,0,3.200000,3.000000,3.000000,2.714286,0.708333,100.000000,10.0
8,548373,A Ashish Reddy,SW Tait,2012-05-18,"Rajiv Gandhi International Stadium, Uppal",Rajasthan Royals,10,0,2.200000,2.666667,5.200000,2.625000,1.500000,107.500000,4.0
9,548376,A Ashish Reddy,R Vinay Kumar,2012-05-20,"Rajiv Gandhi International Stadium, Uppal",Royal Challengers Bangalore,4,1,3.800000,4.500000,5.285714,3.444444,1.400000,156.000000,6.0
10,598000,A Ashish Reddy,AB Dinda,2013-04-05,"Rajiv Gandhi International Stadium, Uppal",Pune Warriors,6,0,4.400000,4.400000,4.500000,3.500000,1.236111,152.500000,1.0


### Bowlers Features

In [18]:
df_bowler['rolling_avg_wickets'] = (
    df_bowler.groupby('bowler')['wickets']
             .rolling(window=5, min_periods=1)
             .mean()
             .shift(1)
             .reset_index(level=0, drop=True)
)


In [19]:
df_bowler['overs_bowled_last5'] = (
    df_bowler.groupby('bowler')['overs_bowled']
             .rolling(window=5, min_periods=1)
             .mean()
             .shift(1)
             .reset_index(level=0, drop=True)
)


In [20]:
df_bowler['venue_wicket_rate'] = (
    df_bowler.groupby(['bowler', 'venue'])['wickets']
             .expanding()
             .mean()
             .shift(1)
             .reset_index(level=[0,1], drop=True)
)


In [21]:
df_bowler['bowler_career_avg'] = (
    df_bowler.groupby('bowler')['wickets']
             .expanding()
             .mean()
             .shift(1)
             .reset_index(level=0, drop=True)
)




In [22]:
df_bowler[
    ['rolling_avg_wickets','overs_bowled_last5',
     'venue_wicket_rate','bowler_career_avg']
] = df_bowler[
    ['rolling_avg_wickets','overs_bowled_last5',
     'venue_wicket_rate','bowler_career_avg']
].fillna(0)


In [23]:
df_bowler['next_match_wicket'] = (
    df_bowler.groupby('bowler')['wickets']
             .shift(-1)
)

df_bowler = df_bowler.dropna()


##### Feature Selection for the ML model (**)


In [24]:
features_batsman = df_batter[
    ['rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg','rolling_strike_rate_10']
]

labels_batsman = df_batter['next_match_runs']


features_bowler = df_bowler[
    ['rolling_avg_wickets',
     'overs_bowled_last5',
     'venue_wicket_rate',
     'bowler_career_avg']
]

labels_bowler = df_bowler['next_match_wicket']



In [25]:
df_batter = df_batter.sort_values('date').reset_index(drop=True)



#### Batsman split

In [26]:
df_batter = df_batter.sort_values('date').reset_index(drop=True)

split_b = int(len(df_batter) * 0.8)

Xb_train = features_batsman.iloc[:split_b]
Xb_test  = features_batsman.iloc[split_b:]

yb_train = labels_batsman.iloc[:split_b]
yb_test  = labels_batsman.iloc[split_b:]


#### Bowler split

In [27]:
df_bowler = df_bowler.sort_values('date').reset_index(drop=True)

split_w = int(len(df_bowler) * 0.8)

Xw_train = features_bowler.iloc[:split_w]
Xw_test  = features_bowler.iloc[split_w:]

yw_train = labels_bowler.iloc[:split_w]
yw_test  = labels_bowler.iloc[split_w:]


In [28]:
batter_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

batter_pipeline.fit(Xb_train)
joblib.dump(batter_pipeline, "../scripts/batter_feature_pipeline.pkl")


['../scripts/batter_feature_pipeline.pkl']

In [29]:
bowler_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

bowler_pipeline.fit(Xw_train)
joblib.dump(bowler_pipeline, "../scripts/bowler_feature_pipeline.pkl")


['../scripts/bowler_feature_pipeline.pkl']

In [31]:
final_batter_df = df_batter[
    [
        'batter','date','venue','bowling_team',
        'rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg','rolling_strike_rate_10',
        'next_match_runs',
    ]
]

final_batter_df.to_csv(
    "../data/cleaned/batter_dataset.csv",
    index=False
)


In [32]:
final_bowler_df = df_bowler[
    [
        'bowler','date','venue','bowling_team',
        'rolling_avg_wickets','overs_bowled_last5',
        'venue_wicket_rate','bowler_career_avg',
        'next_match_wicket'
    ]
]

final_bowler_df.to_csv(
    "../data/cleaned/bowler_dataset.csv",
    index=False
)
